In [1]:
import gmsh
import subprocess
import math
import sys,os, shutil
import pandas
import numpy as np
gmsh.__file__
import matplotlib.pyplot as plt

In [2]:
# GMSH meshing
# export files for mesh and image of mesh/model
def save_mesh(context, meshpath, gifpath):  
    if  not os.path.isdir(context['project_dir']):
        print(f"Creating project dir: `{context['project_dir']}`")
        print( os.mkdir(context['project_dir']))
        
    if gifpath:
        gmsh.fltk.initialize()
        gmsh.write(os.path.join(context['project_dir'], gifpath))
        gmsh.fltk.finalize()        
    if meshpath:
        gmsh.option.setNumber('Mesh.SaveAll', 1)
        gmsh.write(os.path.join(context['project_dir'],meshpath))

# SU2 Case setup
# build a config for a Solver to run a case:        
def save_config(context, base_config_path='base.cfg'):
    config_path=os.path.join(context['project_dir'], f"{context['project_name']}.cfg")
    project_config=open(base_config_path).read().format(**context)
    print(f"writing CNF: '{config_path}'")
    with open(config_path,'w') as f:
        f.write(project_config)

# build a case - geometry, mesh, physical context, solver params. Maybe should be factored a little...
def create_sim(project_name, fuselage_rad=gmsh.pi/16.0, wing_rad=gmsh.pi/4.0, wedge_rad=gmsh.pi/16.0, 
               meshing=True, 
                xshift=0, camX=30.0,camY=60.0,camZ=0.0,camZoom=5.0,
                meshpath=None, gifpath=None, popup=True, angle_of_attack=0.0, meshsize_large=5.0,
              clip_wing=True, case_index=None, case_name=None, project_dir=None):

    # Setup Geometry 
    p_height = 11.0
    p_width = 8.5
    p_thickness = 0.2
    
    gmsh.initialize()
    gmsh.model.add(project_name)
    gmsh.option.setNumber("General.Trackball", 0)

    gmsh.option.setNumber("General.RotationX", camX) #187.3729455233209))
    gmsh.option.setNumber("General.RotationY", camY) #276.7547707531851))
    gmsh.option.setNumber("General.RotationZ", camZ) # 296.8673379509896))
    gmsh.option.setNumber("General.ScaleX", camZoom) 
    gmsh.option.setNumber("General.ScaleY", camZoom) 
    gmsh.option.setNumber("General.ScaleZ", camZoom) 

    # Actual model!
    phi = fuselage_rad # wedge_radians #gmsh.pi/16.0

    # x coordinate of wing fold
    fuselage_height = p_height* math.sin(phi)

    # right vertical wedge; build then tilt; RF Right Fuselage [a,b,c]
    rfb = [0.0, 0.0, 0.0] # back corner of fuselage; origin
    rfa = [0.0, fuselage_height, 0.0]
    rfc = [p_height, 0.0, 0.0] # tip of front
    
    # right wing, vertical
    rwb = rfa.copy()
    rwa = rwb.copy()
    rwa[1] += p_width/2.0 - fuselage_height

    rwc = rfc.copy()

    rf=make_triangle(rfa, rfb, rfc, 0.2)
    rw=make_triangle(rwa, rwb, rwc, 0.2)    
    
    # trim wing
    if clip_wing:
        cutbox=gmsh.model.occ.addBox(
           rwa[0], p_width/2.0-rfa[1], rfa[2]-5*p_thickness,
           p_height,   p_width, 10*p_thickness )   
        rw, v1b = gmsh.model.occ.cut(rw,[(3,cutbox)], removeTool=True)

    #rotate wing along top edge of fuselage
    gmsh.model.occ.rotate(rw,rfa[0],rfa[1],rfa[2],rfc[0]-rfa[0],rfc[1]-rfa[1],rfc[2]-rfa[2],wing_rad)
    
    # merge fuselage and wing
    rh, rhb = gmsh.model.occ.fuse(rf,rw)
    
    #rotate halfplane along bottom of plane
    gmsh.model.occ.rotate(rh,rfb[0],rfb[1],rfb[2],rfc[0]-rfb[0],rfc[1]-rfb[1],rfc[2]-rfb[2],wedge_rad)
        
    lf=make_triangle(rfa, rfb, rfc, 0.2)
    lw=make_triangle(rwa, rwb, rwc, 0.2)
    
    if clip_wing:
        cutbox2=gmsh.model.occ.addBox(
            rwa[0], p_width/2.0-rfa[1], rfa[2]-5*p_thickness,
           p_height,   p_width, 10.0*p_thickness )   
        lw, v1b = gmsh.model.occ.cut(lw,[(3,cutbox2)])
    
    #rotate wing along top edge of fuselage
    gmsh.model.occ.rotate(lw,rfa[0],rfa[1],rfa[2],rfc[0]-rfa[0],rfc[1]-rfa[1],rfc[2]-rfa[2],-wing_rad)
    
    # merge fuselage and wing
    #print(f"Fuselage: {rf} Wing: {rw}")
    lh, lhb = gmsh.model.occ.fuse(lf,lw)
    
    #rotate halfplane along bottom of plane
    gmsh.model.occ.rotate(lh,rfb[0],rfb[1],rfb[2],rfc[0]-rfb[0],rfc[1]-rfb[1],rfc[2]-rfb[2],-wedge_rad)
    
    p1, p1b =  gmsh.model.occ.fuse(lh,rh)
    
    # build skybox
    b1=gmsh.model.occ.addBox(-4*p_height,-4*p_width,-4*p_width, 8*p_height, 8*p_width, 8*p_width)

    # create negative volume around plane model
    v1, v1b = gmsh.model.occ.cut([(3,b1)],p1)

    gmsh.model.occ.synchronize()

    if meshing:
        lcar_big = meshsize_large
        lcar2 = 1.0
        lcar3 = .055

        # Assign a mesh size to all the points:
        gmsh.model.mesh.setSize(gmsh.model.getEntities(0), lcar_big)
        #gmsh.model.mesh.setSize([(2,s) for s in wall_surfaces], lcar1)
        gmsh.model.mesh.generate(3)

    plane_surfaces = []
    wall_surfaces = []

    surfaces = gmsh.model.occ.getEntities(dim=2)
    for s in surfaces:
        com = gmsh.model.occ.getCenterOfMass(s[0], s[1])
        if np.allclose(com, [p_height/2.0, 0, 0],atol=6.0):
            plane_surfaces.append(s[1])
        else:
            wall_surfaces.append(s[1])

    # Set up physical groups for boundaries and markers in Simulation        
    wall_marker = 1
    plane_marker = 2
    gmsh.model.addPhysicalGroup(2, wall_surfaces, wall_marker)
    gmsh.model.setPhysicalName(2, wall_marker, "Walls")
    gmsh.model.addPhysicalGroup(2, plane_surfaces, plane_marker)
    gmsh.model.setPhysicalName(2, plane_marker, "Plane")

    save_mesh(locals(),meshpath, gifpath)
    save_config(context=locals())
    if popup:
        gmsh.fltk.run()
    gmsh.finalize()

# SU2 Geometry
# Build a triangular volume    
def make_triangle(rp0,rp1,rp2,d, mesh_size=1.0, recenter=True):
    #print(f"making TRI: {rp0} | {rp1} | {rp2}...")
    p1 = gmsh.model.occ.addPoint(rp0[0], rp0[1], rp0[2], mesh_size)
    p2 = gmsh.model.occ.addPoint(rp1[0], rp1[1], rp1[2], mesh_size)
    p3 = gmsh.model.occ.addPoint(rp2[0], rp2[1], rp2[2], mesh_size)
    l1 = gmsh.model.occ.addLine(p1,p2)
    l2 = gmsh.model.occ.addLine(p2,p3)
    l3 = gmsh.model.occ.addLine(p3,p1)
    c1 = gmsh.model.occ.addCurveLoop([l1,l2,l3])
    s1 = gmsh.model.occ.addPlaneSurface([c1])
    v1 = gmsh.model.occ.extrude([(2,s1)], 0.0,0.0,d)
    newVolTag = [x for x in v1 if x[0]==3]
    if recenter:
         gmsh.model.occ.translate(newVolTag, 0, 0, -d/2.0)
    #
    return newVolTag

# SU2 Execution
def run_conf(conf, i, meshing, run, as_row=True, popup=False, extract=['lift', 'drag']):
    case_name=f"{project_name}-{i:0>3}"
    #i = conf['case_index']
    #meshsize = conf['mesh_size']
    meshpath=f"{case_name}.su2"
    gifpath=f"{project_name}-{i:0>3}.jpg"
    create_sim(case_name, meshing=meshing, 
        gifpath=gifpath, 
        meshpath=meshpath if meshing else None,
        popup=popup, #meshsize_large=meshsize,
        **conf)
    if run:
        subprocess.run(["/Users/scot/Projects/SU2/bin/SU2_CFD", f"{conf['project_dir']}/{case_name}.cfg"]) 
    run_dict =  {'meshpath': meshpath, 'gifpath':gifpath, 
                 'surface_flow_path': f"{case_name}-surface_flow.csv"}
    if extract:
        sdf=pandas.read_csv( os.path.join(conf['project_dir'],run_dict['surface_flow_path']))
        extracts = {
            'lift':float(sdf[['Momentum_y']].sum()),
            'drag':float(sdf[['Momentum_x']].sum())
        }
        run_dict.update(extracts)
    
    if as_row:
        return pandas.Series([run_dict[c] for c in ['meshpath', 'gifpath', 'surface_flow_path','lift','drag']])
    else:
        return run_dict
    
    
def run_confs(conf_list, meshing, run, meshsize=5.0, popup=False, animation=False):

    for i in range(len(conf_list)):
        run_conf(conf_list[i], i, meshing)

    if animation:
        ffmpeg_path='/Users/scot/bin/ffmpeg'
        subprocess.run([ffmpeg_path, "-r", "6", "-i", f"/Users/scot/Projects/{project_name}-%03d.jpg", f"{project_name}.gif"]) 

# Create and run simulations
### Create sequence of cases

Isn't this where we make a dataframe? Yes. Yes it is.

Dataframes should be (only) information we wouldn't change to run a simulation on a different platform.
So, things like design parameters and case names.
Not things like paths.
Really? 

So things like `project_path` are metadata, associated with the DF not rows. Any given platform will want to build full paths itself with case anmes and such.

In [14]:
project_name = "f5"
project_dir = project_name
df = pandas.DataFrame([{
    "meshsize_large": 5.0,
    "case_index": 5*i + j,
    "case_name": f"{project_name}-{5*i+j:0>3}",
    "project_dir": project_dir,
    "wedge_rad" : gmsh.pi /6.0,
    "wing_rad" : gmsh.pi/2.0,
    "fuselage_rad" : gmsh.pi/48.0 + 0.02*i,
    "angle_of_attack" : -5.0 - j
        } for i in range(5) for j in range(5)])


In [17]:
d2r=2.0*gmsh.pi/360.0
2.0*d2r

0.03490658503988659

In [15]:
# Functions to setup and run case (SU2 is implicit)
# build cases with no mesh or sim
geometry_only = lambda x: run_conf(x,x['case_index'], False, False, extract=True)
# mesh and simulate
full_sim = lambda x: run_conf(x,x['case_index'], True, True)

In [16]:

df[['mesh', 'image', 'surface', 'lift', 'drag']] = df.apply(full_sim, axis=1)
#df[['mesh', 'image', 'surface', 'lift', 'drag']] = df.apply(geometry_only, axis=1)

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 10%] Meshing curve 3 (Line)
Info    : [ 10%] Meshing curve 4 (Line)
Info    : [ 10%] Meshing curve 5 (Line)
Info    : [ 10%] Meshing curve 6 (Line)
Info    : [ 10%] Meshing curve 7 (Line)
Info    : [ 10%] Meshing curve 8 (Line)
Info    : [ 10%] Meshing curve 9 (Line)
Info    : [ 10%] Meshing curve 10 (Line)
Info    : [ 10%] Meshing curve 11 (Line)
Info    : [ 20%] Meshing curve 12 (Line)
Info    : [ 20%] Meshing curve 13 (Line)
Info    : [ 20%] Meshing curve 14 (Line)
Info    : [ 20%] Meshing curve 15 (Line)
Info    : [ 20%] Meshing curve 16 (Line)
Info    : [ 20%] Meshing curve 17 (Line)
Info    : [ 20%] Meshing curve 18 (Line)
Info    : [ 20%] Meshing curve 19 (Line)
Info    : [ 20%] Meshing curve 20 (Line)
Info    : [ 20%] Meshing curve 21 (Line)
Info    : [ 20%] Meshing curve 22 (Line)
Info    : [ 30%] Meshing curve 23 (Line)
Info    : [ 30%] Meshing curve 24 (Line)
I

Info    : 521 edge swaps, 109 node relocations (volume = 406903): worst = 0.000439047 / average = 0.784146 (Wall 0.0140169s, CPU 0.014337s)
Info    : 526 edge swaps, 141 node relocations (volume = 406903): worst = 0.00043963 / average = 0.784155 (Wall 0.0182858s, CPU 0.018268s)
Info    : 0.00 < quality < 0.10 :       136 elements
Info    : 0.10 < quality < 0.20 :        33 elements
Info    : 0.20 < quality < 0.30 :        12 elements
Info    : 0.30 < quality < 0.40 :       345 elements
Info    : 0.40 < quality < 0.50 :       562 elements
Info    : 0.50 < quality < 0.60 :       948 elements
Info    : 0.60 < quality < 0.70 :      2226 elements
Info    : 0.70 < quality < 0.80 :      4718 elements
Info    : 0.80 < quality < 0.90 :      7814 elements
Info    : 0.90 < quality < 1.00 :      3807 elements
Info    : Done optimizing mesh (Wall 0.0288418s, CPU 0.028345s)
Info    : 4194 nodes 24380 elements
Creating project dir: `f5`
None


-------------------------------------------------------
Version       : 4.11.1
License       : GNU General Public License
Build OS      : MacOSARM-sdk
Build date    : 20221221
Build host    : gmsh.info
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blas[petsc] Blossom Cgns DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack Lapack[petsc] MathEx[contrib] Med Mesh Metis[contrib] Mpeg Netgen ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom PETSc Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TouchBar Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.4.0
PETSc version : 3.17.4 (real arithmtic)
OCC version   : 7.6.1
MED version   : 4.1.0
Packaged by   : geuzaine
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------
Info    : Writing 'f5/f5-000.jpg'...
Info    : Done writing 'f5/f5-000.jpg'
Info    : Writing 'f5/

+-----------------------------------------------------------------------------+
|  Inner_Iter|   Time(sec)|    rms[Rho]|   rms[RhoE]|          CL|          CD|
+-----------------------------------------------------------------------------+
|           0|  8.9040e-02|   -1.703714|   -0.682524|   -0.080480|    0.058005|
|           1|  9.0363e-02|   -2.017308|   -0.999006|   -0.083424|    0.058947|
|           2|  8.9333e-02|   -2.096662|   -1.076796|   -0.065779|    0.045478|
|           3|  8.8593e-02|   -2.194087|   -1.169987|   -0.054142|    0.037851|
|           4|  8.8210e-02|   -2.336296|   -1.311315|   -0.048628|    0.035254|
|           5|  8.7914e-02|   -2.453384|   -1.428127|   -0.044876|    0.033994|
|           6|  8.7750e-02|   -2.554307|   -1.528598|   -0.041461|    0.033046|
|           7|  8.7599e-02|   -2.664861|   -1.638736|   -0.038638|    0.032413|
|           8|  8.7476e-02|   -2.783591|   -1.756782|   -0.036749|    0.032061|
|           9|  8.7352e-02|   -2.897222|

Info    : It. 1500 - 1028 nodes created - worst tet radius 1.3202 (nodes removed 402 70)
Info    : It. 2000 - 1447 nodes created - worst tet radius 1.18273 (nodes removed 480 73)
Info    : It. 2500 - 1896 nodes created - worst tet radius 1.08959 (nodes removed 531 73)
Info    : It. 3000 - 2300 nodes created - worst tet radius 1.02441 (nodes removed 615 85)
Info    : 3D refinement terminated (4194 nodes total):
Info    :  - 14 Delaunay cavities modified for star shapeness
Info    :  - 710 nodes could not be inserted
Info    :  - 21016 tetrahedra created in 0.0743944 sec. (282494 tets/s)
Info    : Done meshing 3D (Wall 0.125344s, CPU 0.123821s)
Info    : Optimizing mesh...
Info    : Optimizing volume 2
Info    : Optimization starts (volume = 406903) with worst = 2.92491e-07 / average = 0.770271:
Info    : 0.00 < quality < 0.10 :       229 elements
Info    : 0.10 < quality < 0.20 :       177 elements
Info    : 0.20 < quality < 0.30 :       239 elements
Info    : 0.30 < quality < 0.40 :   

...

Info    : Done writing 'f5/f5-023.jpg'
Info    : Writing 'f5/f5-023.su2'...
Info    : Writing 20169 elements and 4128 nodes
Info    : Done writing 'f5/f5-023.su2'
writing CNF: 'f5/f5-023.cfg'

-------------------------------------------------------------------------
|    ___ _   _ ___                                                      |
|   / __| | | |_  )   Release 7.5.1 "Blackbird"                         |
|   \__ \ |_| |/ /                                                      |
|   |___/\___//___|   Suite (Computational Fluid Dynamics Code)         |
|                                                                       |
-------------------------------------------------------------------------
| SU2 Project Website: https://su2code.github.io                        |
|                                                                       |
| The SU2 Project is maintained by the SU2 Foundation                   |
| (http://su2foundation.org)                                       

         see https://su2code.github.io/docs_v7/Build-SU2-Linux-MacOS/#compiler-optimizations
         see https://su2code.github.io/docs_v7/Build-SU2-Linux-MacOS/#compiler-optimizations
+-----------------------------------------------------------------------------+
|  Inner_Iter|   Time(sec)|    rms[Rho]|   rms[RhoE]|          CL|          CD|
+-----------------------------------------------------------------------------+
|           0|  8.5301e-02|   -1.609761|   -0.588572|   -0.116424|    0.133908|
|           1|  8.5533e-02|   -1.817767|   -0.797192|   -0.128064|    0.145658|
|           2|  8.6530e-02|   -1.981508|   -0.961319|   -0.109299|    0.124219|
|           3|  8.7587e-02|   -2.080820|   -1.057661|   -0.093795|    0.107344|
|           4|  8.7796e-02|   -2.168497|   -1.143742|   -0.085102|    0.098770|
|           5|  8.7803e-02|   -2.293891|   -1.267627|   -0.080257|    0.094508|
|           6|  8.7929e-02|   -2.440411|   -1.412395|   -0.077445|    0.092757|
|           7|

Info    : It. 3000 - 1710 nodes created - worst tet radius 5.78016 (nodes removed 1210 80)
Info    : It. 3500 - 2080 nodes created - worst tet radius 1.04704 (nodes removed 1337 83)
Info    : 3D refinement terminated (4128 nodes total):
Info    :  - 9 Delaunay cavities modified for star shapeness
Info    :  - 1473 nodes could not be inserted
Info    :  - 20526 tetrahedra created in 0.082874 sec. (247677 tets/s)
Info    : Done meshing 3D (Wall 0.137938s, CPU 0.134634s)
Info    : Optimizing mesh...
Info    : Optimizing volume 2
Info    : Optimization starts (volume = 406904) with worst = 7.35838e-06 / average = 0.771252:
Info    : 0.00 < quality < 0.10 :       227 elements
Info    : 0.10 < quality < 0.20 :       149 elements
Info    : 0.20 < quality < 0.30 :       242 elements
Info    : 0.30 < quality < 0.40 :       357 elements
Info    : 0.40 < quality < 0.50 :       560 elements
Info    : 0.50 < quality < 0.60 :       906 elements
Info    : 0.60 < quality < 0.70 :      2140 elements
In

Info    : Done writing 'f5/f5-024.jpg'
Info    : Writing 'f5/f5-024.su2'...
Info    : Writing 20169 elements and 4128 nodes
writing CNF: 'f5/f5-024.cfg'
Info    : Done writing 'f5/f5-024.su2'

-------------------------------------------------------------------------
|    ___ _   _ ___                                                      |
|   / __| | | |_  )   Release 7.5.1 "Blackbird"                         |
|   \__ \ |_| |/ /                                                      |
|   |___/\___//___|   Suite (Computational Fluid Dynamics Code)         |
|                                                                       |
-------------------------------------------------------------------------
| SU2 Project Website: https://su2code.github.io                        |
|                                                                       |
| The SU2 Project is maintained by the SU2 Foundation                   |
| (http://su2foundation.org)                                       

         see https://su2code.github.io/docs_v7/Build-SU2-Linux-MacOS/#compiler-optimizations
         see https://su2code.github.io/docs_v7/Build-SU2-Linux-MacOS/#compiler-optimizations
         see https://su2code.github.io/docs_v7/Build-SU2-Linux-MacOS/#compiler-optimizations
+-----------------------------------------------------------------------------+
|  Inner_Iter|   Time(sec)|    rms[Rho]|   rms[RhoE]|          CL|          CD|
+-----------------------------------------------------------------------------+
|           0|  8.7823e-02|   -1.596902|   -0.575712|   -0.129633|    0.138265|
|           1|  9.0719e-02|   -1.807067|   -0.786640|   -0.144891|    0.150855|
|           2|  9.1077e-02|   -1.973019|   -0.952971|   -0.125752|    0.129152|
|           3|  9.1299e-02|   -2.072516|   -1.049331|   -0.108976|    0.111983|
|           4|  9.0070e-02|   -2.160696|   -1.135850|   -0.099166|    0.103185|
|           5|  8.9116e-02|   -2.285469|   -1.259158|   -0.093623|    0.098799|
|

In [18]:
df.to_csv("f5/f5_df.csv")

In [5]:
df = pandas.read_csv("f4_df.csv")

In [19]:
%matplotlib notebook
plt.scatter(df[['fuselage_rad']], df[['lift']])
plt.show()

ValueError: s must be a scalar, or float array-like with the same size as x and y

In [20]:
%matplotlib notebook
from mpl_toolkits import mplot3d
fig = plt.figure(figsize =(10, 10))
ax = plt.axes(projection ='3d')
 
# Creating plot
ax.scatter(df[['angle_of_attack']],df[['fuselage_rad']], df[['lift']])
ax.set_xlabel("Angle of Attack")
ax.set_ylabel("Fuselage Fold Angle")
ax.set_zlabel("Lift")
ax.set_title("Paper Airplane Design Space")
dir(ax)
plt.show()

<IPython.core.display.Javascript object>

In [ ]:
df

## Animate design "slice" in geometry space
it would be nice to move along various paths through this design space (eg, hold AoA and move the Fuselage Angle across an interesting range) and visualize the change in design and flow with a simple animation.

We'll define the slive of the DF we want, then extract the individual case outputs for them. We can generate the viz for each individually, then export them to frames in an animation.

This is weird because my Paraview python and my Jupyter/GMSH python aren't friends. So, we read a base script to do the viz we want, parametrize in any needed values from the DF, then write out a new python file ready to be run by 

In [21]:
def visualize_case(project_dir, case_name, base_vis_path="para.py"):
    
    base_vis = open(base_vis_path).read()
    #lame_name_hack:
    #proj, idx = case_name.split('_',1)
    #case_name = f"{proj}-{idx:0>3}"
    # interpolated_vis = base_vis.format(locals())
    # case_vis_path=path.join(project_dir, f"{case_name}-para.py")
    #with open(case_vis_path,'w') as f:
    #    f.write(interpolated_vis)
    pvpath = "/Applications/ParaView-5.11.0.app/Contents/bin/pvpython" # para.py fold-004
    print(f"Vis proc for {case_name}...")
    subprocess.run([pvpath, base_vis_path, f"{project_dir}/{case_name}"])
    return f"/Users/scot/Projects/{project_dir}/{case_name}-para.png"
    
vis_path = lambda x: visualize_case(x['project_dir'], x['case_name'])

def animate(project_name, file_paths, output_dir, framerate="2"):
    ffmpeg_path='/Users/scot/bin/ffmpeg'
    for i,file_path in enumerate(file_paths):
        shutil.copyfile(file_path, os.path.join(output_dir, f"animation_frame-{i:0>3}.png"))
    subprocess.run([ffmpeg_path, "-r", framerate,  "-y", "-i", 
                    f"/Users/scot/Projects/{project_name}/animation_frame-%03d.png", 
                    f"/Users/scot/Projects/{project_name}/{project_name}.gif"]) 
    #for i,file_path in enumerate(file_paths):
    #    os.remove(os.path.join(output_dir, f"animation_frame_{i}.png"))
    return f"/Users/scot/Projects/{project_name}/{project_name}.gif"

### run visualizations and set path to vis outputs

In [22]:
df['vis_path'] = df.apply(vis_path,axis=1)

Vis proc for f5-000...
para-processing f5/f5-000...


(   1.963s) [paraview        ]    vtkPVRenderView.cxx:3273  WARN| vtkPVRenderView (0x7fba4c81fc40): Refusing to enable OSPRay because it is not supported running in this configuration.


Vis proc for f5-001...
para-processing f5/f5-001...


(   1.929s) [paraview        ]    vtkPVRenderView.cxx:3273  WARN| vtkPVRenderView (0x7fafa1f2c7c0): Refusing to enable OSPRay because it is not supported running in this configuration.


Vis proc for f5-002...
para-processing f5/f5-002...


...

(   1.785s) [paraview        ]    vtkPVRenderView.cxx:3273  WARN| vtkPVRenderView (0x7fd23cffb9a0): Refusing to enable OSPRay because it is not supported running in this configuration.


Vis proc for f5-023...
para-processing f5/f5-023...


(   1.815s) [paraview        ]    vtkPVRenderView.cxx:3273  WARN| vtkPVRenderView (0x7fd97675a630): Refusing to enable OSPRay because it is not supported running in this configuration.


Vis proc for f5-024...
para-processing f5/f5-024...


(   1.788s) [paraview        ]    vtkPVRenderView.cxx:3273  WARN| vtkPVRenderView (0x7fde4d7ce190): Refusing to enable OSPRay because it is not supported running in this configuration.


In [13]:

animate('f4', list(df.loc[df['angle_of_attack']==3.0][['vis_path']]['vis_path']), output_dir='f4')

ffmpeg version N-112441-gc06d3d2404-tessus  https://evermeet.cx/ffmpeg/  Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 11.0.0 (clang-1100.0.33.17)
  configuration: --cc=/usr/bin/clang --prefix=/opt/ffmpeg --extra-version=tessus --enable-avisynth --enable-fontconfig --enable-gpl --enable-libaom --enable-libass --enable-libbluray --enable-libdav1d --enable-libfreetype --enable-libgsm --enable-libmodplug --enable-libmp3lame --enable-libmysofa --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenh264 --enable-libopenjpeg --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvmaf --enable-libvo-amrwbenc --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs --enable-libxml2 --enable-libxvid --enable-libzimg --enable-libzmq --enable-libzvbi --enable-version3 --pkg-conf

'/Users/scot/Projects/f4/f4.gif'

In [ ]:
visualize_case('f2', 'f2_3')

In [ ]:
ffmpeg_path='/Users/scot/bin/ffmpeg'
subprocess.run([ffmpeg_path, "-r", "6", "-i", 
                f"/Users/scot/Projects/{project_name}/{project_name}-%03d.jpg", 
                f"/Users/scot/Projects/{project_name}{project_name}.gif"]) 

### Make exploratory (unmeshed) animation of models

In [ ]:
df['geometry'] = run_confs(conf_list, False, False)

### Generate meshes
would be nice to leave the animation alone...

In [ ]:
for i in range(len(conf_list)):
    case_name=f"{project_name}-{i:0>3}"
    create_sim(case_name, meshing=True, 
                gifpath=f"{project_name}_meshed-{i:0>3}.jpg", 
                meshpath=f"{case_name}.su2",
                popup=False, meshsize_large=2.0,
                **conf_list[i])
    #subprocess.run(["/Users/scot/Projects/SU2/bin/SU2_CFD", f"{case_name}.cfg"]) 
ffmpeg_path='/Users/scot/bin/ffmpeg'
subprocess.run([ffmpeg_path, "-i", f"/Users/scot/Projects/{project_name}_meshed-%03d.jpg", f"{project_name}_meshed.gif"]) 

## Actually run cases!

In [ ]:
for i in range(len(conf_list)):
    case_name=f"{project_name}-{i:0>3}"
    create_sim(case_name, meshing=True, 
                gifpath=None, #f"{project_name}-{i:0>3}.jpg", 
                meshpath=f"{case_name}.su2",
                popup=False, meshsize_large=4.0,
                **conf_list[i])
    subprocess.run(["/Users/scot/Projects/SU2/bin/SU2_CFD", f"{case_name}.cfg"]) 
#ffmpeg_path='/Users/scot/bin/ffmpeg'
#subprocess.run([ffmpeg_path, "-i", f"/Users/scot/Projects/{project_name}-%03d.jpg", f"{project_name}_meshed.gif"]) 

In [ ]:
import subprocess
        # subprocess.call("ffmpeg -i t8-%d.jpg t8.mpg".split(' '))
# /Users/scot/bin/ffmpeg -i /Users/scot/Projects/pplane-%03d.jpg test.gif
ffmpeg_path='/Users/scot/bin/ffmpeg'
subprocess.run([ffmpeg_path, "-i", f"/Users/scot/Projects/{project_name}-%03d.jpg", f"{project_name}.gif"]) 

### Superlight config formatting

In [ ]:
base_config_path='base.cfg'
config_path=f"{project_name}.cfg"
project_config=open(base_config_path).read().format(**globals())
with open(config_path,'w') as f:
    f.write(project_config)
#base_config

## Run subprocess
* Used to need this for `gmsh` file translation, but no longer.
* Now is this just to run SU2? Fine, but likely replaceable with a direct SU2 API....

In [ ]:
import subprocess

subprocess.run(["/Users/scot/Projects/SU2/bin/SU2_CFD", config_path]) 

## Read field output from Simulation run

In [ ]:
import pandas
for i in range(10):
    sdf=pandas.read_csv(f"fold-{i:0>3}-surface_flow.csv")
    print(f"{conf_list[i]['fuselage_rad']}: {df[['Momentum_y']].sum()} {df[['Momentum_x']].sum()}")

In [ ]:
sys.path.append('/Applications/ParaView-5.11.0.app/Contents/Python/')
import paraview.simple
# '/Applications/ParaView-5.11.0.app/Contents/Python/

In [ ]:
ffmpeg_path='/Users/scot/bin/ffmpeg'
subprocess.run([ffmpeg_path, "-r", "6", "-i", f"/Users/scot/Projects/{project_name}-%03d-para.png", f"{project_name}-para.gif"]) 